In [1]:
from pathlib import Path
import prody as pr
import multiprocess

def get_sequence(path):
    structure = pr.parsePDB(str(path))
    return {'laser_path': path, 'sequence': structure.ca.getSequence()}


all_paths = [pdb for pdb in Path('./laser_outputs/').glob('*/*.pdb')]

all_data = []
with multiprocess.Pool(28) as p:
    for result in p.imap_unordered(get_sequence, all_paths):
        all_data.append(result)


In [2]:
import pandas as pd
df = pd.DataFrame(all_data)
ligand_smiles = 'COC1=CC=C(N2C3=C(C(C(N)=O)=N2)CCN(C4=CC=C(N5CCCCC5=O)C=C4)C3=O)C=C1'

boltz_inputs_path = Path('./boltz_inputs/')
boltz_inputs_path.mkdir(exist_ok=True)

def get_boltz_yaml_boilerplate(sequence: str, smiles: str, predict_affinity: bool):
    boltz_yaml = f"version: 1\nsequences:\n  - protein:\n      id: A\n      sequence: {sequence}\n      msa: empty\n  - ligand:\n      id: B\n      smiles: '{smiles}'\n"
    if predict_affinity:
        boltz_yaml += "properties:\n  - affinity:\n      binder: B\n"
    return boltz_yaml

for i, row in df.iterrows():
    # with open(boltz_inputs_path / f'{i}.yaml', 'w') as f:
    #     f.write(get_boltz_yaml_boilerplate(row['sequence'], ligand_smiles, predict_affinity=False))
    pass

In [3]:
df

laser_path  \
0      laser_outputs/cluster_24_sup-cluster_90_41/des...   
1      laser_outputs/cluster_44_sup-cluster_39_233/de...   
2      laser_outputs/cluster_29_sup-cluster_61_45/des...   
3      laser_outputs/cluster_04_sup-cluster_6_94/desi...   
4      laser_outputs/cluster_05_sup-cluster_36_119/de...   
...                                                  ...   
14995  laser_outputs/cluster_06_sup-cluster_15_60/des...   
14996  laser_outputs/cluster_46_sup-cluster_38_24/des...   
14997  laser_outputs/cluster_46_sup-cluster_38_24/des...   
14998  laser_outputs/cluster_46_sup-cluster_38_24/des...   
14999  laser_outputs/cluster_47_sup-cluster_59_85/des...   

                                                sequence  
0      MSEEEGRQLGEAILAAFNNKDLEALKKLAGEDVKLYVKTTNTEYTT...  
1      ATEEDIAAVKKLVKEFIEAMNNRDLDKILEMLDPNARALTGNKDFE...  
2      SEPEVKAEVTQLIKDFVKAVNDGDVDAIRALLAPDCTGTLLSTNKT...  
3      GAPEAIKAAVSKVITDLVNALNNGDVDAAMSLLDPNAKFYNKNANK...  
4      DAEEKKAAVKKLITDFIEAMNKGDIDKIKEIGDPDAKGTLKNTGKT...  
...                                                  ...  
14995  LTEKEVREGVEEFVKAINEGDLDKLAEMADPNAVGVLPDGKVLTSR...  
14996  LTKEELTKAATELIKDFLAACDAGDIDKIRGMLAEDARVTALGTKK...  
14997  MTKEEKTKAATELIKEFLAACDAGDIEKVRSLLAEDAKVTAKGTNK...  
14998  LTKAEKTAAATALIKEFLASCDAGDIDRVRSLLAEDATVTAKGTNK...  
14999  LTEEEKKQATIDIIEQCYDALNAGDLEACKSLFADDAVVTLPDGTV...  

[15000 rows x 2 columns]

In [4]:
df.iloc[13160]

laser_path    laser_outputs/cluster_38_sup-cluster_23_50/des...
sequence      PTEEQIKEWIKKFHEAIENKDLDKLLELVQDDTVGVASDGTVCTTK...
Name: 13160, dtype: object